In [9]:
import pandas as pd
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 80)
pd.set_option('display.max_rows', 1000)

In [2]:
def get_dataframe(name):
    
    file_name = f'Output/{name}.csv'
    return pd.read_csv(file_name)

dataset = get_dataframe('dataset_filtered')
dataset_ind = get_dataframe('individual record')
dataset_dis = get_dataframe('district record')
display(dataset['Detected State'].unique())

array(['Kerala', 'Delhi', 'Telangana', 'Haryana', 'Uttar Pradesh',
       'Ladakh', 'Tamil Nadu', 'Jammu and Kashmir', 'Karnataka',
       'Maharashtra', 'Punjab', 'Rajasthan', 'Andhra Pradesh',
       'Uttarakhand', 'Odisha', 'Puducherry', 'West Bengal', 'Chandigarh',
       'Chhattisgarh', 'Gujarat', 'Himachal Pradesh', 'Madhya Pradesh',
       'Bihar', 'Manipur', 'Mizoram', 'Goa',
       'Andaman and Nicobar Islands', 'Jharkhand', 'Assam',
       'Arunachal Pradesh', nan, 'Tripura', 'Meghalaya',
       'Dadra and Nagar Haveli and Daman and Diu', 'State Unassigned',
       'Sikkim', 'Nagaland', 'Lakshadweep'], dtype=object)

In [3]:
def get_template():
    '''Returns an empty dataframe with required columns'''
    
    columns = ['Day Id', 'Total Number of cases', 'Total Number of fatalities',
               'Total Number of cases(Females)', 'Total Number of cases(Males)',
               'Total Number of fatalities(Females)', 'Total Number of fatalities(Males)',
               
               'Age (1-10)', 'Age (11-20)', 'Age (21-30)', 'Age (31-40)', 'Age (41-50)', 
               'Age (51-60)', 'Age (61-70)', 'Age (71-80)', 'Age (81 and above)',
               
               'Age (1-10) Female', 'Age (11-20) Female', 'Age (21-30) Female', 'Age (31-40) Female', 'Age (41-50) Female', 
               'Age (51-60) Female', 'Age (61-70) Female', 'Age (71-80) Female', 'Age (81 and above) Female',
               
               'Age (1-10) Male', 'Age (11-20) Male', 'Age (21-30) Male', 'Age (31-40) Male', 'Age (41-50) Male', 
               'Age (51-60) Male', 'Age (61-70) Male', 'Age (71-80) Male', 'Age (81 and above) Male',
               
               'Age (1-10) Fatalities', 'Age (11-20) Fatalities', 'Age (21-30) Fatalities', 'Age (31-40) Fatalities', 'Age (41-50) Fatalities', 
               'Age (51-60) Fatalities', 'Age (61-70) Fatalities', 'Age (71-80) Fatalities', 'Age (81 and above) Fatalities',
               
               'Age (1-10) Female Fatalities', 'Age (11-20) Female Fatalities', 'Age (21-30) Female Fatalities', 'Age (31-40) Female Fatalities', 'Age (41-50) Female Fatalities', 
               'Age (51-60) Female Fatalities', 'Age (61-70) Female Fatalities', 'Age (71-80) Female Fatalities', 'Age (81 and above) Female Fatalities',
               
               'Age (1-10) Male Fatalities', 'Age (11-20) Male Fatalities', 'Age (21-30) Male Fatalities', 'Age (31-40) Male Fatalities', 'Age (41-50) Male Fatalities', 
               'Age (51-60) Male Fatalities', 'Age (61-70) Male Fatalities', 'Age (71-80) Male Fatalities', 'Age (81 and above) Male Fatalities',
                     
              ]
    
    return pd.DataFrame(columns=columns)

In [4]:
def each_row(day_from_first, state=None):
    if state:

        dataset_local = dataset[dataset['Detected State'] == state]
        dataset_ind_local = dataset_ind[dataset_ind['Detected State'] == state]
        dataset_dis_local = dataset_dis[dataset_dis['Detected State'] == state]
        
    df_total = dataset_local[dataset_local['Day Id'] == day_from_first]  # Required dataframe from full dataset
    df_ind = dataset_ind_local[dataset_ind_local['Day Id'] == day_from_first] # Required dataframe from district record dataset
    df_dis = dataset_dis_local[dataset_dis_local['Day Id'] == day_from_first] # Required dataframe from individual level dataset
    
    df_total_d = dataset_local[dataset_local['Status Day Id'] == day_from_first]
    df_ind_d = dataset_ind_local[dataset_ind_local['Status Day Id'] == day_from_first]
    df_dis_d = dataset_dis_local[dataset_dis_local['Status Day Id'] == day_from_first]
    
    def get_cases(df, gender=None):
        df = df[df['Current Status'] == 'Hospitalized']
        
        if gender:
            return df[df['Gender'] == gender].shape[0]
        return df['Num Cases'].sum()
    
    def get_fatalities(df, gender=None):
        df = df[df['Current Status'] == 'Deceased']
     
        if gender:
            return df[df['Gender'] == gender].shape[0]
        return df['Num Cases'].sum()
    
    def get_cases_distribution(df, lower, upper, gender=None):
        lower = str(lower); upper = str(upper)
        df = df[df['Current Status'] == 'Hospitalized']
        df['Age Bracket'] = df['Age Bracket'].apply(str)
        df = df[(df['Age Bracket'] >= lower) & (df['Age Bracket'] <= upper)]

        if gender:
            return df[df['Gender'] == gender].shape[0]
        return df.shape[0]

    def get_fatalities_distribution(df, lower, upper, gender=None):
        lower = str(lower); upper = str(upper)
        df = df[df['Current Status'] == 'Deceased']
        df['Age Bracket'] = df['Age Bracket'].apply(str)
        df = df[(df['Age Bracket'] >= lower) & (df['Age Bracket'] <= upper)]
        
        if gender:
            return df[df['Gender'] == gender].shape[0]
        return df.shape[0]
        
    new_row = dict()
    new_row['Day Id'] = day_from_first
    new_row['Total Number of cases'] = get_cases(df_total)
    new_row['Total Number of cases(Males)'] = get_cases(df_ind, 'M')
    new_row['Total Number of cases(Females)'] = get_cases(df_ind, 'F')
    new_row['Total Number of fatalities'] = get_fatalities(df_total_d)
    new_row['Total Number of fatalities(Males)'] = get_fatalities(df_ind_d, 'M')
    new_row['Total Number of fatalities(Females)'] = get_fatalities(df_ind_d, 'F')
    
    for i in range(1, 81, 10):
        new_row[f'Age ({i}-{i+9})'] = get_cases_distribution(df_total, i, i+9)
        new_row[f'Age ({i}-{i+9}) Female'] = get_cases_distribution(df_ind, i, i+9, 'F')        
        new_row[f'Age ({i}-{i+9}) Male'] = get_cases_distribution(df_ind, i, i+9, 'M')
    
    new_row['Age (81 and above)'] = get_cases_distribution(df_total, 81, 120)
    new_row['Age (81 and above) Female'] = get_cases_distribution(df_ind, 81, 120, 'F')
    new_row['Age (81 and above) Male'] = get_cases_distribution(df_ind, 81, 120, 'M')
    
    
    for i in range(1, 81, 10):
        new_row[f'Age ({i}-{i+9}) Fatalities'] = get_fatalities_distribution(df_total_d, i, i+9)
        new_row[f'Age ({i}-{i+9}) Female Fatalities'] = get_fatalities_distribution(df_ind_d, i, i+9, 'F')        
        new_row[f'Age ({i}-{i+9}) Male Fatalities'] = get_fatalities_distribution(df_ind_d, i, i+9, 'M')
    
    new_row['Age (81 and above) Fatalities'] = get_fatalities_distribution(df_total_d, 81, 120)
    new_row['Age (81 and above) Female Fatalities'] = get_fatalities_distribution(df_ind_d, 81, 120, 'F')
    new_row['Age (81 and above) Male Fatalities'] = get_fatalities_distribution(df_ind_d, 81, 120, 'M')
 

    return new_row

# each_row(100)

In [11]:
def append_row():
#     day_id = dataset['Day Id'].unique()
    day_id = [x for x in range(0, 476)]

    processed_df = get_template()
    for day in day_id:
        processed_df = processed_df.append(each_row(day, 'Bihar'), ignore_index=True, sort=False)
    display(processed_df)
    
    return processed_df
    
# combined_df = append_row()

,Day Id,Total Number of cases,Total Number of fatalities,Total Number of cases(Females),Total Number of cases(Males),Total Number of fatalities(Females),Total Number of fatalities(Males),Age (1-10),Age (11-20),Age (21-30),Age (31-40),Age (41-50),Age (51-60),Age (61-70),Age (71-80),Age (81 and above),Age (1-10) Female,Age (11-20) Female,Age (21-30) Female,Age (31-40) Female,Age (41-50) Female,Age (51-60) Female,Age (61-70) Female,Age (71-80) Female,Age (81 and above) Female,Age (1-10) Male,Age (11-20) Male,Age (21-30) Male,Age (31-40) Male,Age (41-50) Male,Age (51-60) Male,Age (61-70) Male,Age (71-80) Male,Age (81 and above) Male,Age (1-10) Fatalities,Age (11-20) Fatalities,Age (21-30) Fatalities,Age (31-40) Fatalities,Age (41-50) Fatalities,Age (51-60) Fatalities,Age (61-70) Fatalities,Age (71-80) Fatalities,Age (81 and above) Fatalities,Age (1-10) Female Fatalities,Age (11-20) Female Fatalities,Age (21-30) Female Fatalities,Age (31-40) Female Fatalities,Age (41-50) Female Fatalities,Age (51-60) Female Fatalities,Age (61-70) Female Fatalities,Age (71-80) Female Fatalities,Age (81 and above) Female Fatalities,Age (1-10) Male Fatalities,Age (11-20) Male Fatalities,Age (21-30) Male Fatalities,Age (31-40) Male Fatalities,Age (41-50) Male Fatalities,Age (51-60) Male Fatalities,Age (61-70) Male Fatalities,Age (71-80) Male Fatalities,Age (81 and above) Male Fatalities
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,471.0,400.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
472,472.0,370.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
473,473.0,385.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
474,474.0,347.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [1]:
def save_to_csv(df, name):
    path = f'Output/{name}.csv'
    df.to_csv(path, index=False)

# save_to_csv(combined_df, 'Bihar_summary')

In [12]:
display(combined_df.head(100))

,Day Id,Total Number of cases,Total Number of fatalities,Total Number of cases(Females),Total Number of cases(Males),Total Number of fatalities(Females),Total Number of fatalities(Males),Age (1-10),Age (11-20),Age (21-30),Age (31-40),Age (41-50),Age (51-60),Age (61-70),Age (71-80),Age (81 and above),Age (1-10) Female,Age (11-20) Female,Age (21-30) Female,Age (31-40) Female,Age (41-50) Female,Age (51-60) Female,Age (61-70) Female,Age (71-80) Female,Age (81 and above) Female,Age (1-10) Male,Age (11-20) Male,Age (21-30) Male,Age (31-40) Male,Age (41-50) Male,Age (51-60) Male,Age (61-70) Male,Age (71-80) Male,Age (81 and above) Male,Age (1-10) Fatalities,Age (11-20) Fatalities,Age (21-30) Fatalities,Age (31-40) Fatalities,Age (41-50) Fatalities,Age (51-60) Fatalities,Age (61-70) Fatalities,Age (71-80) Fatalities,Age (81 and above) Fatalities,Age (1-10) Female Fatalities,Age (11-20) Female Fatalities,Age (21-30) Female Fatalities,Age (31-40) Female Fatalities,Age (41-50) Female Fatalities,Age (51-60) Female Fatalities,Age (61-70) Female Fatalities,Age (71-80) Female Fatalities,Age (81 and above) Female Fatalities,Age (1-10) Male Fatalities,Age (11-20) Male Fatalities,Age (21-30) Male Fatalities,Age (31-40) Male Fatalities,Age (41-50) Male Fatalities,Age (51-60) Male Fatalities,Age (61-70) Male Fatalities,Age (71-80) Male Fatalities,Age (81 and above) Male Fatalities
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
def get_template_extended():
    '''Returns an empty dataframe with required columns'''
    
    columns = ['State', 'Total Number of cases', 'Total Number of fatalities',
               'Total Number of cases(Females)', 'Total Number of cases(Males)',
               'Total Number of fatalities(Females)', 'Total Number of fatalities(Males)',
               
               'Age (1-10)', 'Age (11-20)', 'Age (21-30)', 'Age (31-40)', 'Age (41-50)', 
               'Age (51-60)', 'Age (61-70)', 'Age (71-80)', 'Age (81 and above)',
               
               'Age (1-10) Female', 'Age (11-20) Female', 'Age (21-30) Female', 'Age (31-40) Female', 'Age (41-50) Female', 
               'Age (51-60) Female', 'Age (61-70) Female', 'Age (71-80) Female', 'Age (81 and above) Female',
               
               'Age (1-10) Male', 'Age (11-20) Male', 'Age (21-30) Male', 'Age (31-40) Male', 'Age (41-50) Male', 
               'Age (51-60) Male', 'Age (61-70) Male', 'Age (71-80) Male', 'Age (81 and above) Male',
               
               'Age (1-10) Fatalities', 'Age (11-20) Fatalities', 'Age (21-30) Fatalities', 'Age (31-40) Fatalities', 'Age (41-50) Fatalities', 
               'Age (51-60) Fatalities', 'Age (61-70) Fatalities', 'Age (71-80) Fatalities', 'Age (81 and above) Fatalities',
               
               'Age (1-10) Female Fatalities', 'Age (11-20) Female Fatalities', 'Age (21-30) Female Fatalities', 'Age (31-40) Female Fatalities', 'Age (41-50) Female Fatalities', 
               'Age (51-60) Female Fatalities', 'Age (61-70) Female Fatalities', 'Age (71-80) Female Fatalities', 'Age (81 and above) Female Fatalities',
               
               'Age (1-10) Male Fatalities', 'Age (11-20) Male Fatalities', 'Age (21-30) Male Fatalities', 'Age (31-40) Male Fatalities', 'Age (41-50) Male Fatalities', 
               'Age (51-60) Male Fatalities', 'Age (61-70) Male Fatalities', 'Age (71-80) Male Fatalities', 'Age (81 and above) Male Fatalities',
                     
              ]
    
    return pd.DataFrame(columns=columns)

In [28]:
def get_row_extended(state):

    df_ind = dataset_ind[dataset_ind['Detected State'] == state]
    df_total = dataset[dataset['Detected State'] == state]
    
    def get_cases(df, gender=None):

        if gender:
            return df[df['Gender'] == gender].shape[0]
        return df['Num Cases'].sum()

    def get_fatalities(df, gender=None):
        df = df[df['Current Status'] == 'Deceased']
     
        if gender:
            return df[df['Gender'] == gender].shape[0]
        return df['Num Cases'].sum()
    
    def get_cases_distribution(df, lower, upper, gender=None):
        lower = str(lower); upper = str(upper)
#         df = df[df['Current Status'] == 'Hospitalized']
        df['Age Bracket'] = df['Age Bracket'].apply(str)
        df = df[(df['Age Bracket'] >= lower) & (df['Age Bracket'] <= upper)]

        if gender:
            return df[df['Gender'] == gender].shape[0]
        return df.shape[0]

    def get_fatalities_distribution(df, lower, upper, gender=None):
        lower = str(lower); upper = str(upper)
        df = df[df['Current Status'] == 'Deceased']
        df['Age Bracket'] = df['Age Bracket'].apply(str)
        df = df[(df['Age Bracket'] >= lower) & (df['Age Bracket'] <= upper)]
        
        if gender:
            return df[df['Gender'] == gender].shape[0]
        return df.shape[0]
        
    new_row = dict()
    new_row['State'] = state
    new_row['Total Number of cases'] = get_cases(df_total)
    new_row['Total Number of cases(Males)'] = get_cases(df_ind, 'M')
    new_row['Total Number of cases(Females)'] = get_cases(df_ind, 'F')
    new_row['Total Number of fatalities'] = get_fatalities(df_total)
    new_row['Total Number of fatalities(Males)'] = get_fatalities(df_ind, 'M')
    new_row['Total Number of fatalities(Females)'] = get_fatalities(df_ind, 'F')
    
    for i in range(1, 81, 10):
        new_row[f'Age ({i}-{i+9})'] = get_cases_distribution(df_total, i, i+9)
        new_row[f'Age ({i}-{i+9}) Female'] = get_cases_distribution(df_ind, i, i+9, 'F')        
        new_row[f'Age ({i}-{i+9}) Male'] = get_cases_distribution(df_ind, i, i+9, 'M')
    
    new_row['Age (81 and above)'] = get_cases_distribution(df_total, 81, 120)
    new_row['Age (81 and above) Female'] = get_cases_distribution(df_ind, 81, 120, 'F')
    new_row['Age (81 and above) Male'] = get_cases_distribution(df_ind, 81, 120, 'M')
    
    
    for i in range(1, 81, 10):
        new_row[f'Age ({i}-{i+9}) Fatalities'] = get_fatalities_distribution(df_total, i, i+9)
        new_row[f'Age ({i}-{i+9}) Female Fatalities'] = get_fatalities_distribution(df_ind, i, i+9, 'F')        
        new_row[f'Age ({i}-{i+9}) Male Fatalities'] = get_fatalities_distribution(df_ind, i, i+9, 'M')
    
    new_row['Age (81 and above) Fatalities'] = get_fatalities_distribution(df_total, 81, 120)
    new_row['Age (81 and above) Female Fatalities'] = get_fatalities_distribution(df_ind, 81, 120, 'F')
    new_row['Age (81 and above) Male Fatalities'] = get_fatalities_distribution(df_ind, 81, 120, 'M')
 
    return new_row
    
# get_row_extended('Bihar')

In [30]:
def get_stats():
    states = dataset['Detected State'].unique()
    processed_df = get_template_extended()
    for state in states:
        processed_df = processed_df.append(get_row_extended(state), ignore_index=True, sort=False)
    
    display(processed_df)
    return processed_df
    
processed_df = get_stats()

,State,Total Number of cases,Total Number of fatalities,Total Number of cases(Females),Total Number of cases(Males),Total Number of fatalities(Females),Total Number of fatalities(Males),Age (1-10),Age (11-20),Age (21-30),Age (31-40),Age (41-50),Age (51-60),Age (61-70),Age (71-80),Age (81 and above),Age (1-10) Female,Age (11-20) Female,Age (21-30) Female,Age (31-40) Female,Age (41-50) Female,Age (51-60) Female,Age (61-70) Female,Age (71-80) Female,Age (81 and above) Female,Age (1-10) Male,Age (11-20) Male,Age (21-30) Male,Age (31-40) Male,Age (41-50) Male,Age (51-60) Male,Age (61-70) Male,Age (71-80) Male,Age (81 and above) Male,Age (1-10) Fatalities,Age (11-20) Fatalities,Age (21-30) Fatalities,Age (31-40) Fatalities,Age (41-50) Fatalities,Age (51-60) Fatalities,Age (61-70) Fatalities,Age (71-80) Fatalities,Age (81 and above) Fatalities,Age (1-10) Female Fatalities,Age (11-20) Female Fatalities,Age (21-30) Female Fatalities,Age (31-40) Female Fatalities,Age (41-50) Female Fatalities,Age (51-60) Female Fatalities,Age (61-70) Female Fatalities,Age (71-80) Female Fatalities,Age (81 and above) Female Fatalities,Age (1-10) Male Fatalities,Age (11-20) Male Fatalities,Age (21-30) Male Fatalities,Age (31-40) Male Fatalities,Age (41-50) Male Fatalities,Age (51-60) Male Fatalities,Age (61-70) Male Fatalities,Age (71-80) Male Fatalities,Age (81 and above) Male Fatalities
0,Kerala,5093571.0,11208.0,3503,5631,3428,5486,5,25,114,306,694,1436,2165,1934,0,2,10,34,105,221,540,757,741,0,3,12,69,188,462,888,1401,1187,0,3,9,57,250,662,1407,2148,1930,0,2,6,20,94,215,528,751,738,0,1,3,37,156,446,875,1393,1186,0
1,Delhi,2558530.0,22000.0,9,35,1,2,0,0,6,4,2,1,2,0,0,0,0,4,1,0,0,1,0,0,0,0,2,3,2,1,1,0,0,0,0,0,1,0,0,2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0
2,Telangana,1096973.0,3297.0,9,35,2,13,2,2,12,4,6,8,6,2,0,0,1,0,1,3,0,2,0,0,2,1,12,3,2,8,4,2,0,2,0,0,0,2,4,4,2,0,0,0,0,0,1,0,0,0,0,2,0,0,0,1,4,4,2,0
3,Haryana,1429882.0,8623.0,11,5,1,1,0,0,6,1,1,8,2,0,0,0,0,4,1,0,1,1,0,0,0,0,2,0,1,0,1,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
4,Uttar Pradesh,3117776.0,19981.0,26,33,1,0,0,6,7,16,6,4,2,1,0,0,4,2,4,3,2,1,1,0,0,2,5,12,3,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,Ladakh,37016.0,186.0,1,4,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,Tamil Nadu,4403084.0,28970.0,7060,11818,67,139,85,1359,3949,3714,3026,2318,1142,533,0,38,578,1557,1212,1024,808,437,206,0,42,749,2275,2394,1921,1455,690,321,0,0,1,4,11,26,45,51,36,0,0,1,1,3,7,19,17,7,0,0,0,3,8,19,26,34,29,0
7,Jammu and Kashmir,576295.0,3993.0,4,10,1,1,1,0,4,1,1,2,4,0,0,0,0,0,1,0,0,2,0,0,1,0,4,0,1,2,2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
8,Karnataka,5035838.0,31028.0,25846,45056,1808,3920,466,4913,14533,14455,11834,10106,6493,2884,0,216,2187,5848,4948,3825,3464,2274,1004,0,250,2726,8683,9506,8007,6641,4217,1880,0,2,19,115,334,734,1326,1526,882,0,2,9,44,87,230,431,457,282,0,0,10,71,247,504,895,1068,600,0
9,Maharashtra,10900410.0,108491.0,31,49,2,13,0,2,12,11,14,13,9,2,0,0,0,6,3,5,3,4,0,0,0,2,6,8,6,9,5,2,0,0,0,1,0,2,5,6,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,2,5,5,1,0


In [33]:
save_to_csv(processed_df, 'State wise distribution')